In this notebook, I investigate the dynamics of the fixed points to the phase locked solutions of the TbWB model when the mean synapses are slowly varying. In this problem, it is typically not the case that $\phi^y=0$ is  stable for this system. The dynamics of the mean synapses dominate $\phi^z$, we reduce the dynamics to just $d\phi^x/d\tau$. The resulting equations are

$$\frac{d\phi^x}{d\tau} = \frac{\alpha^{xx}(\tau)}{2}\left[H^{xx}\left(- \phi^x,\tau\right) - H^{xx}\left(\phi^x,\tau\right)\right] + \alpha^{xy}(\tau)\left[H^{xy}\left(- \phi^x+\phi^z,\tau\right) - H^{xy}\left(\phi^z,\tau\right) \right]$$

$$\phi^z(\tau) \approx \phi^z(0) + \int_0^\tau [\bar s^x(\tau') - \bar s](\beta^{yx}-\beta^{xx})/\varepsilon + [\bar s^y(\tau') - \bar s](\beta^{yy} - \beta^{xy})/\varepsilon d\tau'$$

In [ ]:
import thetaslowmod_phase
import numpy as np
import matplotlib.pylab as mp
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
from scipy.optimize import brentq


a1=.5;b1=7.;c1=6.5
a2=1.1;b2=25.;c2=25.1

p = thetaslowmod_phase.Phase(T=0,
                             thxin=[0,0],
                             thyin=[0,0],
                             sx0=1,
                             sy0=1,
                             a1=a1,b1=b1,c1=c1,
                             a2=a2,b2=b2,c2=c2,
                             muy=5.43,
                             eps=0.01
                            )

T = 5
dt = .0025
TN = int(T/dt)
t = np.linspace(0,T,TN)

mean_sol = p.run_mean_field(1.02,1.02,T,dt,no_eps=True)

if False:
    mp.figure()
    mp.plot(t,mean_sol[:,0])
    mp.plot(t,mean_sol[:,1])

bxx = p.beta_xx_coeff
bxy = p.beta_xy_coeff
byx = p.beta_yx_coeff
byy = p.beta_yy_coeff

sbar = p.sbarx
pz = 0.0

zero_vals = np.zeros(TN)
antiphase_vals = np.zeros(TN)
px_vals = np.zeros(TN)
pz_vals = np.zeros(TN)

px_vals[0] = .05
pz_vals[0] = pz

fig = plt.figure()
ax11 = fig.add_subplot(221)
ax12 = fig.add_subplot(222)
ax21 = fig.add_subplot(223)
ax22 = fig.add_subplot(224)

for i in range(TN):
    sx = p.sxa_fn(t[i])
    sy = p.sya_fn(t[i])
    
    freqx = np.sqrt(p.a1+p.b1*sx-p.c1*sy)
    freqy = np.sqrt(p.a2+p.b2*sx-p.c2*sy)
    
    Tx = 1./freqx
    Ty = 1./freqy
    
    dom11,tot11 = p.generate_h(sx,sy,choice='xx',return_domain=True)
    dom22,tot22 = p.generate_h(sx,sy,choice='yy',return_domain=True)

    dom12,tot12 = p.generate_h_inhom(sx,sy,choice='xy',return_domain=True)
    dom21,tot21 = p.generate_h_inhom(sx,sy,choice='yx',return_domain=True)
    

    # create linear interp
    hxx = interp1d(dom11,tot11)
    hxy = interp1d(dom12,tot12)
    hyx = interp1d(dom21,tot21)
    hyy = interp1d(dom22,tot22)
    
    pz = pz + dt*( (sx-p.sbarx)*(byx-bxx)/p.eps + (sy-p.sbary)*(byy-bxy)/p.eps)
    
    px_rhs = .5*(hxx(np.mod(-dom11,Tx))-hxx(np.mod(dom11,Tx))+hxy(mp.mod(-dom11+pz,Tx))-hxy(np.mod(pz,Tx)))
    
    if i%100 == 0:
        ax11.plot(hxx(np.mod(-dom11,Tx))-hxx(np.mod(dom11,Tx)),color=str(1.*i/TN))
        ax12.plot(hxy(mp.mod(-dom11+pz,Tx))-hxy(np.mod(pz,Tx)),color=str(1.*i/TN))
        #ax12.plot(dom11,hxx(np.mod(dom11,Tx)),color=str(1.*i/TN))
        ax21.plot(hxy(mp.mod(dom11,Tx)),color=str(1.*i/TN))
        ax22.plot(px_rhs,color=str(1.*i/TN))
    
    
    # create interpolation of right hand side
    px_rhs_fn1 = interp1d(dom11,px_rhs)
    
    # create modulus interpolation of right hand side
    def px_rhs_fn2(x):
        x = np.mod(x,Tx)
        return px_rhs_fn1(x)
    
    if i < TN-1:
        px_vals[i+1] = px_vals[i] + dt*px_rhs_fn2(px_vals[i])
    pz_vals[i] = np.mod(pz+Ty/2.,Ty)-Ty/2.
    # find zero
    maxper = Tx
    minper = 0
    halfper = Tx/2.
    margin = halfper*.9
    xlo = halfper - margin
    xhi = halfper + margin
    
    zero_vals[i] = brentq(px_rhs_fn2,xlo,xhi)
    antiphase_vals[i] = Tx/2.
    
    #mp.plot(dom11,px_rhs)
"""
mp.figure()
mp.plot(t,zero_vals)
mp.plot(t,antiphase_vals,color='gray')
mp.plot(t,-antiphase_vals,color='gray')
mp.plot(t,px_vals)
mp.plot(t,pz_vals)
#mp.xlim(0,1.1)

mp.figure()
for i in range(1,len(mean_sol[:,0])):
    mp.plot([mean_sol[:,0][i-1],mean_sol[:,0][i]],[mean_sol[:,1][i-1],mean_sol[:,1][i]])
"""
mp.show()
mp.close()


loading or recomputing beta functions
Running mean field
loading or recomputing h functions
Running mean field
